# Of anagrams, probability, entropy, and tyPhon ;)

I found Exercise  43 in [Torbjorn Lager](https://www.gu.se/en/about/find-staff/torbjornlager)'s list of [46 Python exercises](https://github.com/PabloVallejo/python-exercises/blob/master/exercises-list.md) oddly mesmerising, and could not stop playing. Here are a few notes I have put together for the amusement of my Python students and of some of my friends who also enjoy word games, probabilities, entropy, programming, and stuff. The text of the exercise is this:

--------

An anagram is a type of word play, the result of rearranging the letters of a word or phrase to produce a
new word or phrase, using all the original letters exactly once; e.g., orchestra ~ carthorse. Using the word
list  [unixdict.txt](https://raw.githubusercontent.com/quinnj/Rosetta-Julia/master/unixdict.txt), write a program that finds the sets of words that
share the same characters that contain the most words in them.

-------

I have updated the link to the dictionary and uploaded it to this binder, so you can just run the code below (for those of you who are not familiar with [Jupyter Notebooks](https://jupyter.org/): click on the Run button above to run each code cell or on the "Fast Forward" button to run them all, you can edit the notebook and experiment as you wish).


## Solving the exercise

This exercise is actually not that hard, once you realise that you can efficiently check if a word is the anagram of another by sorting their characters:

In [ ]:
def sort_chars(word):
    """ Return a tuple containing the characters of word in alphabetical order """
    wlist=list(word)
    wlist.sort()
    return tuple(wlist)

For instance:

In [ ]:
sort_chars("orchestra")

In [ ]:
sort_chars("carthorse")

We want tuples because we are going to use these as keys in a dictionary for our usual counting trick. A string would do just as well, lists are no good because they are mutable and therefore not hashable.

In [ ]:
groups={} # keys: sorted chars; values: list of words sharing those exact chars
wcount=0
with open("unixdict.txt", "rt") as INFILE:
    for word in INFILE:
        word=word.rstrip() # remove \n
        wkey = sort_chars(word)
        try:
            # if we have seen the key before, add word to the anagram group
            groups[wkey].append(word)
        except KeyError: 
            # never seen wkey before - create new group for it
            groups[wkey]=[word]            
        wcount+=1
    
print(f"{wcount} words processed")

In [ ]:
# this dictionary is big!
groups

So how big is the largest set of words that are the anagram of each other? We can write a comprehension to find out:

In [ ]:
maxsize=max([len(wg) for wg in groups.values()])
maxsize

Let's find those groups of words - there may be more than one:

In [ ]:

largest_groups=[]
for wlist in groups.values():
    if len(wlist)==5:
        largest_groups.append(wlist)

for group in largest_groups:
    print(group)

In fact, there are six. 

That's it for the exercise...

## ... what about the other groups?

I suspect if we consider smaller groups of anagrams, we'll find more. Let's turn the code above into a function and check that:

In [ ]:
def select_group_size(groups, n):
    listed_groups=[]
    for wlist in groups.values():
        if len(wlist)==n:
            listed_groups.append(wlist)
    return listed_groups
    

In [ ]:
select_group_size(groups, 4)

As you'd expect, there are even more groups of three words:

In [ ]:
len(select_group_size(groups, 3))

In fact, we can plot the number of groups as a function of their size. Let's write a little helper object to aid with the counting, it'll come in handy even later:

In [ ]:
class Aggregate:
    """ Aggregate a list of (x,y) pairs passed to the constructor with respect to x """
    
    def __init__(self, data):
        """ Initialise the object by aggregating data wrt the first component of the pairs """
        self._histogram={}
        # do the actual aggregation here
        for (hbin, hval) in data:
            self._histogram[hbin]=self._histogram.get(hbin, 0)+hval
            
    def get(self, hbin):
        """ returns the value associated to a bin, or zero if the bin does not exist """
        return self._histogram.get(hbin, 0)
    
    def low_bin(self):
        """ returns the lowest non-zero bin """
        return min(self._histogram.keys())
    
    def high_bin(self):
        """ return the highest non-zero bin """
        return max(self._histogram.keys())
    
    def get_range(self):
        """ return a range object for integer bins (with missing values filled in),
        a list of sorted bin labels in the other cases """
        try:
            return range(self.low_bin(), self.high_bin()+1)
        except TypeError: # bins are not int
            return sorted(self._histogram.keys())
    
    def get_list(self, hbins=None):
        """ return values on the list of bins passed, or on the entire range
        if no list is passed """
        if hbins==None:
            hbins=self.get_range()
        return [self.get(b) for b in hbins]

Back to computing the number of groups as a function of their size. Let's have each group vote for its size:

In [ ]:
# each group counts for 1
group_sizes=[(len(v), 1) for v in groups.values()]
group_size_histo=Aggregate(group_sizes)

As expected, here is the result - note that groups of size one mean no anagram at all for that word! That's of course the majority.

In [ ]:
group_size_histo.get_list()

We can even plot a nice graph:

In [ ]:
import matplotlib.pyplot as plt
# embed plots in notebook
%matplotlib inline 

In [ ]:
# better leave "singleton" words out or they will dwarf all the rest
bins=range(2, group_size_histo.high_bin()+1)
plt.bar(bins, group_size_histo.get_list(bins), tick_label=bins)
plt.title('Anagram groups vs words they contain')
plt.show()

## ...so what about word length? 

Almost all the anagram groups we have seen consist of words of 4 or 5 letters, a few have 3 letters or 6. So what about other word lengths? What is the longest anagram in the dictionary? Let's try to find out:

In [ ]:
# Only select len(v)>1 because we want the word to have some anagram! 
maxlength= max([len(k) for k,v  in groups.items() if len(v)>1])

In [ ]:
maxlength

Again, we can write a function that finds groups with words of a given length for us:

In [ ]:
def select_word_length(groups, l):
    anagrams=[]
    for k,wg in groups.items():
        if len(k)==l and len(wg)> 1:
            anagrams.append(wg)
    return anagrams

and here it is:

In [ ]:
select_word_length(groups, maxlength)

again, there seem to be more hits for shorter words:

In [ ]:
select_word_length(groups, maxlength-1)

...and even more for even shorter words:

In [ ]:
select_word_length(groups, maxlength-2)

at ```maxlength-3```, we even find a group of three words:

In [ ]:
select_word_length(groups, 9)

(I never suspected ```cartesian``` of being related to ```sectarian```, but that's now ```ascertain```'d).

As before, we can plot a neat little histogram. We can count the groups

In [ ]:
# each groups casts a vote for the length of its words
wlengths=[(len(k),1) for k,v in groups.items() if len(v)>1]
groups_by_wlen=Aggregate(wlengths)

or we can count the individual words:

In [ ]:
# each word in an anagram group casts a vote for its length
wlengths=[(len(k),len(v)) for k,v in groups.items() if  len(v)>1]
anagrams_by_wlen=Aggregate(wlengths)

In [ ]:
bins=range(2, maxlength+1)

ax = plt.subplot(111) # get a handle for the axes
h1=ax.bar([x-0.15 for x in bins], groups_by_wlen.get_list(bins), 
       width=0.3, align='center')
# h1, h2 are handles for the bar series
h2=ax.bar([x+0.15 for x in bins], anagrams_by_wlen.get_list(bins), 
       width=0.3, align='center')
plt.legend([h1,h2], ['groups', 'anagrams'])
plt.title('No of groups/anagrams vs word length')
plt.show()

Either way, we see a clear maximum for words of length 4, followed by words of length 5.
Most anagrams are around this length. Surely that's because a majority of words in the dictionary are of length 4? Let's check:

In [ ]:
# each key corresponds to len(v) words of length len(k). Here even 'singleton' words count
wlength=[(len(k), len(v)) for k,v in groups.items()]
uxdict_by_length=Aggregate(wlength)

bins=range(1, uxdict_by_length.high_bin()+1)
plt.bar(bins, uxdict_by_length.get_list(bins), tick_label=bins)
plt.title('Unixdict words vs length')
plt.show()

Hold on! The most common word length in the dictionary is actually 7 characters. So where are all the groups of 7 or 8 letter long anagrams? Are we missing anything here?

## A simple probabilistic model

This discrepancy must be down to some combinatorial effect of word size. In fact, for two words to be the anagram of each other, they must 
* be of the same length, and
* be one the permutation of the other.

Now the first factor,as we have seen, favours words of length close to 7, but what about the second?

For the second factor, consider the probability that two coins that you throw on the floor at random land on the same tile. You throw one coin first, and note the tile it landed on. Clearly now, the probability that the second coin will land on the same tile is equal to the area of that tile, divided by the size of the room. Now for two words of length $l$, the size of the room is the set of all sequences of length $l$ that can be formed with the 26 letters of the alphabet, that is $26^l$. Given a choice of letters (which is where the first coin lands), the size of the tile corresponds to all possible permutations of such letters, that is $l!$. Thus we can approximate the probability that two words are the anagrams of each other given that both of them are of length $l$ as:
\begin{equation}
P(anagram|l,l)=\frac{l!}{26^l}.
\end{equation}
Of course the probability that they form an anagram pair given that they have different lengths is zero. This model is obviously very crude. For one thing, it treats words purely as random sequences of characters. However, we can as a first approximation assume that the rules that govern word formation apply evenly across the space of sequences, and so do not affect our calculation. Besides, our model assumes all characters are equally likely to occur in words, which is certainly not the case. However, this simple model still captures some important information.  Note in particular that, since all words in the dictionary are shorter than 26 characters, the numerator becomes progressively smaller as compared to the denominator, in the range we are interested in. In fact, $P(anagram|l,l)$ decreases very steeply with word length (note the log scale): <a id='alphabet_size'></a>

In [ ]:
# You may want to come back and change this later, 
# read on for now!
alphabet_size=26

In [ ]:
from math import  factorial

xaxis=range(2,23)
p_anagram_ll=[factorial(l)/alphabet_size**l for l in xaxis]
plt.plot(xaxis, p_anagram_ll, marker='.', linestyle='')
plt.xticks(xaxis)
plt.yscale('log')
plt.title("P(anagram|l,l) vs word length")
plt.show()

this term therefore suggests that, even if longer words are more common, they may have a smaller chance of forming an anagram pair simply because the possible permutations of a word form an increasingly small subset of the space of possible words (interestingly, note that this would not be the case for much longer words or alphabets with fewer letters, or indeed if some characters are used but rarely). We can therefore hope that combining this factor with the probability distribution of word length will lead to a realistic estimate for the probability of anagrams of a given length. 

In detail we have, for the probability of two words forming an anagram pair of length $l$:
\begin{equation}
P(anagram, l, l)=P(anagram|l,l) P(l) P(l)
\end{equation}
where $P(l)$ is the probability that a word has length $l$. This we can estimate from the length distribution of words in our dictionary, as computed above:

In [ ]:
# we are not really interested in words of length 1, so start from 2
ubl=uxdict_by_length.get_list(range(2,23))
# normalise the counts so that probabilities sum to 1
p_l=[l/sum(ubl) for l in ubl]

and therefore:

In [ ]:
# each value in this list is P(anagram,l,l)
p_anagram=[(pa_ll*pl*pl) for pa_ll, pl in zip(p_anagram_ll, p_l)]

In order to obtain a word count, we need to multiply this by twice the number of pair of 
words from the dictionary. Out of $N$ items we can form $N(N-1)/2$ pairs, hence


In [ ]:
# note the first element is l=2, and we are not interested in anything 
# beyond the length of the longest anagram
n_anagrams=[p *wcount*(wcount-1) for p in p_anagram[:maxlength-1]]

we can plot this against the data for comparison:

In [ ]:
import matplotlib.lines as mlines

bins=range(2, maxlength+1)
ax = plt.subplot(111)
h1=ax.bar(bins, anagrams_by_wlen.get_list(bins), color='tab:blue', align='center')
ax.plot(bins, n_anagrams, marker='d', color='tab:orange', linestyle='')
# proxy line with no data for creating the legend - plot returns no usable handle
h2= mlines.Line2D([], [], color='tab:orange', marker='d', linestyle='None')
plt.legend([h1,h2], ['actual', 'model'])
plt.title('No of anagrams vs word length')
plt.show()

As can be seen, the maximum is in the right position, and the general shape of the curve appears to be correct. However, anagrams of length three are wrongly predicted to be more likely than those of length five; also, our model grossly underestimates the number of anagrams. This is really not surprising - we made some coarse approximations, and our model contains no information at all about the language. Rather, it is surprising that simple back-of-the envelope calculations manage to capture at least the general trend!  

## Random dealings in entropy

The fact that our model underestimates the number of anagrams suggests that we may have overestimated the denominator in our combinatorial factor $P(anagram|l,l)$. In fact, not all letters are equally likely to occur in a word. We can estimate the relative frequency of characters in the English language from our dictionary:

In [ ]:
# This is a large comprehension, using round brackets to delimit it turns it into
# a generator, that's computed lazily when needed. Note the nested loops. The final
# if is needed to filter out characters such as digits that also appear in the dictionary.
# Each char in a key is counted once for every word in the corresponding group
charlist=((c, len(v)) for k,v in groups.items() for c in k if c.isalpha())
char_histo=Aggregate(charlist)
all_letters=char_histo.get_range()
char_counts=char_histo.get_list(all_letters)
char_probs=[n/sum(char_counts) for n in char_counts]

In [ ]:
plt.bar(all_letters, char_probs, tick_label=all_letters)
plt.title('Character probability distribution')
plt.show()

As can be seen, 'e' is the most frequent letter in English, followed by 'a'; a few letters such as 'j', 'q', 'x' and 'z' are actually very rare. This suggests that there may be, in some sense, an "effective" alphabet size that's smaller than 26 characters. We can estimate that using [Shannon's entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)), that measures the minimum bit rate required to encode a sequence, given the probability distribution over the characters. In our case, Shannon's entropy can be computed as
\begin{equation}
H=\sum_c P(c)\log_2\left(\frac{1}{P(c)}\right)=-\sum_c P(c)\log_2 P(c)
\end{equation}
where the sum is over all characters and $\log_2 x$ is the logarithm of $x$ in base $2$ (for instance, $\log_2 8 = 3$ since $2^3=8$). Shannon's entropy is measured in bits.

In [ ]:
from math import log2

H=sum([-p*log2(p) for p in char_probs])
H

This value of $H\simeq 4.24\,\mathrm{bit}$ is significantly lower than the entropy of a uniform distribution over an alphabet of $26$ characters, that would be $\log_2 26 \simeq 4.70\,\mathrm{bit}$. Of course, our alphabet really has 26 characters, but they are not uniformly distributed. We can replace it in our model with a hypothetical, uniformly distributed alphabet of smaller size, so that its entropy has the correct value for English text; for that, the number of character needs to be equal to

In [ ]:
2**H

If you now [scroll back up](#alphabet_size) and change ```alphabet_size``` to this "effective size" of $18.87\simeq 19$ characters, and then run the calculations in the previous section again, you will see that this new model, while by no means perfect, fits the data a lot better.

## That's all, folks...

All that's left is finding a nice little anagram for the title:

In [ ]:
query="python" # change this as suits

print ("Anagrams for "+ query)
key=sort_chars(query)
try:
    wl=groups[key]
    if len(wl)>1:
        print([w for w in wl if w!=query])
    else:
        print ("None found!")
except KeyError:
    print ("Word not in wordlist")

(C) 2021 [Fabrizio Smeraldi](http://www.eecs.qmul.ac.uk/~fabri/) - All rights reserved. 